In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('eval_metrics.csv')

# 计算小于对应 rmsd 的比例
rmsd_values = np.arange(0, 5.01, 0.01)  # 0 到 5，步长为 0.01
proportions = [df[df['rmsd'] < rmsd].shape[0] / df.shape[0] for rmsd in rmsd_values]

# 计算每0.1的 rmsd 值
rmsd_values_0_1 = np.arange(0, 5.01, 0.1)
proportions_0_1 = [df[df['rmsd'] < rmsd].shape[0] / df.shape[0] for rmsd in rmsd_values_0_1]

# 绘制浅蓝色的散点图
plt.scatter(rmsd_values_0_1, proportions_0_1, color='lightblue')

# 标注小于2和小于5的纵坐标
plt.annotate(f'Proportion at RMSD < 2: {proportions[int(2/0.01)]}', (2, proportions[int(2/0.01)]))
plt.annotate(f'Proportion at RMSD < 5: {proportions[int(5/0.01)]}', (5, proportions[int(5/0.01)]))

# 在横坐标为2的地方，绘制该点的横坐标和纵坐标线
plt.axvline(x=2, color='gray', linestyle='--')
plt.axhline(y=proportions[int(2/0.01)], color='gray', linestyle='--')

plt.xlabel('RMSD')
plt.ylabel('Proportion')
plt.title('Proportion of values with RMSD less than X')
plt.show()

In [ ]:
import os
cnt = 0
gen = 0
all = 0
with open('test_pdb.csv', 'r') as file:
    lines = file.readlines()[1:]  
    all = len(lines)
    for line in lines:
        pdb_code = line.strip()[-8:-4]  
        # print(pdb_code)
        folder_name = f'PDBBindData/PDBBind_rcsb/{pdb_code}'
        if os.path.exists(folder_name):
            gen += 1
            if len(os.listdir(folder_name)) > 2:
                cnt += 1
print(">2: ", cnt)
print("generated", gen)
print(">2 Percentage: ", cnt/gen)
print("Not generated: ", all - gen)

In [39]:
folder_path = "PDBBindData/PDBBind_processed"

pdb_codes = [name for name in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, name))]
from gemmi import cif
pdb_codes.remove('1a50') # for some reason, this directory is empty

for i in pdb_codes:
    with open(f"PDBBindData/PDBBind_processed/{i}/{i}_protein_processed.pdb", "r") as f:
        lines = f.readlines()
        chains = None
        try:
            chains = lines[0].split("'")[1].replace("chain ", "").split(" or ")
        except:
            continue
        chains_tmp = []
        for idx, chain in enumerate(chains):
            if len(chain) > 1:

                # print(idx, chains)
                las = chain[-1]
                fir = chains[idx - 1][0]
                # print(fir, las)
                
                for t in range(ord(fir) + 1, ord(las) + 1):
                    # print(chr(i))
                    chains_tmp.append(chr(t))
            else:
                chains_tmp.append(chain)
        # print(chains)
        chains = chains_tmp
        # if len(chains) != len(chains_tmp):
        #     print(chains)
        #     print(chains_tmp)
        if len(chains) > 5:
            print(i)
        continue
        cif_path = f'/home/t-kaiyuangao/Workspace/DynamicBind/PDBBindData/.tmp/{i}.cif'
                
        doc = cif.read_file(cif_path)
        block = doc.sole_block()
        list_assembly = list(block.find_values('_pdbx_struct_assembly_gen.asym_id_list'))[0].split(',')
        if list_assembly == ['']:
            continue
            # if chain not in list_assembly:
                # print(i, chain)
        # print(chains)

5jzi
6eiz
3nba
4j70
3ubx
6g8n
2vxa
3m3r
1g6r
3cii
1vea
2vx9
4yvz
4r02
4l4v
2ylc
6g7f
6qm7
1t13
1rri
6g8m
6cha


In [ ]:
cif_path = f'/home/t-kaiyuangao/Workspace/DynamicBind/PDBBindData/.tmp/2uw8.cif'
                
doc = cif.read_file(cif_path)
block = doc.sole_block()
list_assembly = list(block.find_values('_pdbx_struct_assembly_gen.asym_id_list'))[0].split(',')
list_assembly

In [13]:
import pandas as pd
from rdkit import Chem
import os
import requests
from tqdm import tqdm

folder_path = "PDBBindData/PDBBind_processed"
folder_names = [name for name in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, name))]

folder_names.remove('1a50') # for some reason, this directory is empty

import re

for i in tqdm(folder_names):
    folder_name = i
    cif_url = f"https://www.rcsb.org/sequence/{i}"
    cif_path = f"PDBBindData/.tmp/{folder_name}.cif"
    desir = "Small Molecules"
    res_url = requests.get(cif_url)
    # matches = re.findall(desir, res_url.text)
    # if len(matches) == 0:
    #     print(i)
    # print(matches)
    response = requests.get(cif_url)
    if response.status_code == 200:
        continue
    print(f"Not Found {i}")
    if os.path.exists(cif_path):
        continue

        with open(cif_path, 'wb') as f:
            f.write(response.content)
            print("yes")
    else:
        print(f"Error downloading CIF file for {folder_name}")

  0%|          | 0/19119 [00:00<?, ?it/s]

  0%|          | 35/19119 [00:15<2:21:56,  2.24it/s]


KeyboardInterrupt: 

In [ ]:
from gemmi import cif
import threading
import concurrent.futures

lock = threading.Lock()
lock2 = threading.Lock()
cnt = 0
idx = 0

def f(folder_name):
    global cnt
    global idx
    cif_path = f'/home/t-kaiyuangao/Workspace/DynamicBind/PDBBindData/.tmp/{i}.cif'
    doc = cif.read_file(cif_path)
    block = doc.sole_block()
    list_assembly = list(block.find_values('_pdbx_struct_assembly_gen.asym_id_list'))
    if len(list_assembly) > 1:
        cnt += 1
    # with lock2:
    idx += 1
    # print(cnt / idx)

executor = concurrent.futures.ThreadPoolExecutor()

for i in tqdm(folder_names):
    executor.submit(f, i)

executor.shutdown(wait=True)

print(cnt, len(folder_names))
print(idx)


In [1]:
import PDBBindData.test0

In [3]:
import os
PDBBindData.test0.download_mol2("3dx4", "asd")
folder_path = "PDBBindData/PDBBind_processed"
pdb_codes = [name for name in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, name))]
# pdb_codes

In [4]:
removed = []
from tqdm import tqdm
from IPython.display import clear_output
removed.append('4jdf')
removed.append('4pox')
removed.append('1qon')
removed.append('5ab1')
removed.append('1a50')
questioned = []
errored = []

for i in tqdm(pdb_codes):
    if i in removed:
        continue
    status = 0
    try:
        status = PDBBindData.test0.download_mol2(i, "")
    except:
        try:
            status = PDBBindData.test0.download_mol2(i, "")
        except:
            clear_output()
            errored.append(i)
            print(f"Mismatched: {questioned}")
            print(f"Errored twice tried: {errored}")
            continue
    if status != 0:
        clear_output()
        questioned.append(i)
        print(f"Mismatched: {questioned}")
        print(f"Errored: {errored}")

100%|█████████▉| 19068/19120 [40:17<00:03, 16.52it/s]

Mismatched: ['6p7p', '3u0t', '6c5k', '4g8o', '1ow4', '4lkg', '4hki', '3kwb', '4q3u', '1him', '1dzp', '5jzi', '3tpu', '1cea', '1o0d', '3ob0', '3nba', '4l2g', '3l9m', '3kd7', '3n9r', '5jop', '2z5t', '5m63', '4hlg', '3nik', '1jyc', '6dyw', '4zyq', '1ths', '3bt9', '3tvw', '6p7q', '1oay', '1ngw', '2r3c', '1uys', '1w2x', '5un1', '6of5', '2e9v', '5cbm', '4avg', '1mpa', '5bjt', '4q3t', '6nt2', '6ow7', '1fd7', '4uuq', '4fci', '4kzq', '1qvt', '6r18', '4l0s', '3ifp', '5kzp', '3w54', '5nwb', '4hup', '5c4l', '5vnb', '1jqy', '4asj', '1mhw', '2jb6', '4zy1', '5csz', '4hlk', '4hv7', '5jyp', '4loq', '5t2y', '4z07', '3coj', '3lqi', '4l31', '4aa7', '4cku', '2aig', '3ifo', '2fx9', '3ubx', '3zlk', '2ydw', '3btj', '3f81', '1u9l', '6csf', '5nt0', '3kr5', '2yem', '4k7n', '4ory', '4nl1', '4hs6', '3kdm', '4l2f', '6f09', '2zpk', '5nsp', '3b95', '1oau', '1k2i', '2dbl', '1fl3', '4huo', '4hlf', '4kmu', '6bky', '2ntf', '4uzd', '2z5s', '3l9l', '1g6r', '6ee2', '4myh', '6eir', '4l2k', '3o5n', '1y0x', '1etz', '4ec4', '3h

100%|█████████▉| 19073/19120 [40:17<00:02, 20.67it/s]

100%|██████████| 19120/19120 [40:23<00:00,  7.89it/s]


In [3]:
questioned = ['6p7p', '3u0t', '6c5k', '4g8o', '1ow4', '4lkg', '4hki', '3kwb', '4q3u', '1him', '1dzp', '5jzi', '3tpu', '1cea', '1o0d', '3ob0', '3nba', '4l2g', '3l9m', '3kd7', '3n9r', '5jop', '2z5t', '5m63', '4hlg', '3nik', '1jyc', '6dyw', '4zyq', '1ths', '3bt9', '3tvw', '6p7q', '1oay', '1ngw', '2r3c', '1uys', '1w2x', '5un1', '6of5', '2e9v', '5cbm', '4avg', '1mpa', '5bjt', '4q3t', '6nt2', '6ow7', '1fd7', '4uuq', '4fci', '4kzq', '1qvt', '6r18', '4l0s', '3ifp', '5kzp', '3w54', '5nwb', '4hup', '5c4l', '5vnb', '1jqy', '4asj', '1mhw', '2jb6', '4zy1', '5csz', '4hlk', '4hv7', '5jyp', '4loq', '5t2y', '4z07', '3coj', '3lqi', '4l31', '4aa7', '4cku', '2aig', '3ifo', '2fx9', '3ubx', '3zlk', '2ydw', '3btj', '3f81', '1u9l', '6csf', '5nt0', '3kr5', '2yem', '4k7n', '4ory', '4nl1', '4hs6', '3kdm', '4l2f', '6f09', '2zpk', '5nsp', '3b95', '1oau', '1k2i', '2dbl', '1fl3', '4huo', '4hlf', '4kmu', '6bky', '2ntf', '4uzd', '2z5s', '3l9l', '1g6r', '6ee2', '4myh', '6eir', '4l2k', '3o5n', '1y0x', '1etz', '4ec4', '3h91', '4zy0', '3tv5', '3cii', '2fw6', '4l33', '4l0i', '4zx9', '3bti', '4q3r', '3kr4', '3twd', '4l10', '4a1w', '6df2', '1jyi', '4hww', '4b4g', '1axs', '3bki', '3h2o', '1jn2', '4l34', '4kzl', '3lp7', '3t6r', '6pyd', '4fxy', '4b3u', '3wtn', '3e3b', '4zy2', '6nfn', '3h0s', '5ctb', '4hlm', '1bo5', '4b4m', '6mmq', '6dyu', '3nim', '6r1w', '3sjt', '4kn7', '2i47', '5nbw', '3lp4', '1azx', '2or9', '6r13', '4zx8', '2a2x', '5nsx', '3fyk', '3btl', '3v4v', '3btc', '1lt6', '4k3n', '6oos', '1r6z', '4dhl', '4hxq', '4l4v', '4l0t', '3f80', '5vh0', '1qvu', '1fl6', '1h2u', '3nil', '1xzx', '4xt2', '4dsy', '6bl1', '6eee', '2feq', '4b5b', '4u0g', '3n8k', '6qau', '5o87', '6r1c', '5wtt', '2qtr', '4r6t', '4l09', '3ti8', '4kzu', '3tf7', '4e8y', '6eww', '4glr', '3e5u', '5elw', '1tze', '1ibg', '4wht', '5doh', '5ij7', '3e8u', '6dyv', '6o50', '1ml1', '3h0j', '3mvj', '4l0b', '4q3s', '5eoc', '1y3a', '5ctc', '4l32', '4n07', '3h0q', '3t8w', '6bkx', '5u49', '3zst', '6r12', '4l0v', '5y7z', '3fyj', '6c5f', '1wc1', '6bt0', '5nvc', '1br8', '6epy', '6o4w', '3ogm', '6mia', '4hlh', '6cse', '4kn4', '1oar', '4jfx', '6pf4']
errored = ['3buo', '3bum', '1xn3', '3bux', '3buw']

In [4]:
import shutil
import os


destination_folder = 'questioned'

# Create the destination folder if it doesn't exist
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)
for i in questioned:
    source_file = f'PDBBindData/PDBBind_processed/{i}/{i}_protein_processed.pdb'
    # Copy the file to the destination folder
    shutil.copy(source_file, destination_folder)


In [50]:
import shutil
import os


destination_folder = 'errored'
errored = tmp
# Create the destination folder if it doesn't exist
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)
for i in errored:
    source_file = f'PDBBindData/PDBBind_processed/{i}/{i}_protein_processed.pdb'
    # Copy the file to the destination folder
    shutil.copy(source_file, destination_folder)

In [51]:
import shutil

# 创建一个 zip 归档文件
shutil.make_archive('errored', 'zip', 'errored')

'/home/t-kaiyuangao/Workspace/DynamicBind/errored.zip'

In [7]:
import shutil

# 创建一个 zip 归档文件
shutil.make_archive('questioned', 'zip', 'questioned')

'/home/t-kaiyuangao/Workspace/DynamicBind/questioned.zip'

In [8]:
len(questioned)

260

In [77]:
i = '1u9l'
import os


destination_folder = f'PDBBindData/PDBBind_rcsb/{i}'

if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

source_file = f'PDBBindData/PDBBind_processed/{i}/{i}_ligand.mol2'
# Copy the file to the destination folder
shutil.copy(source_file, destination_folder)

source_file = f'PDBBindData/PDBBind_processed/{i}/{i}_protein_processed.pdb'
# Copy the file to the destination folder
shutil.copy(source_file, destination_folder)


'PDBBindData/PDBBind_rcsb/1u9l/1u9l_protein_processed.pdb'

In [69]:
import requests
url = "https://models.rcsb.org/v1/1ow4/ligand?auth_seq_id=1003&label_asym_id=E&encoding=mol2&filename=1ow4_E_2AN.mol2"

response = requests.get(url)
lig_file = 'PDBBindData/PDBBind_rcsb/1ow4/2AN_E.mol2' # 注意改pdbcode
with open(lig_file, 'w') as f:
    f.write(response.text)

In [81]:
from PDBBindData import test0
test_cp("1uys", True)

{}
chains:  ['A', 'B']
dic_name:  {}


0

In [47]:
print(dic_name)

NameError: name 'dic_name' is not defined

In [80]:
import re
import requests
import pandas as pd
import os
import threading

from gemmi import cif
import shutil
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import concurrent.futures

cnt = 0
success_cnt = 0
tot = 0

lock = threading.Lock()
lock_cnt = threading.Lock()
df = pd.DataFrame()

def test_cp(pdb_code, test_mode=False):
    cif_path = f'/home/t-kaiyuangao/Workspace/DynamicBind/PDBBindData/.tmp/{pdb_code}.cif'
    doc = cif.read_file(cif_path)
    block = doc.sole_block()
    # Find assembly Chains
    list_assembly_all = list(block.find_values('_pdbx_struct_assembly_gen.asym_id_list'))
    list_assembly_all = [_.split(',') for _ in list_assembly_all]
    checked = 0
    flag = False
    if len(list_assembly_all) > 1:
        # Means that there are multiple assemblies,
        # Need information from the pdbbind website
        sequence_url = f"https://www.rcsb.org/annotations/{pdb_code}"
        resp_seq = requests.get(sequence_url)
        # The annotations website guarantees that there is only protein information
        pattern_auth = re.compile(r'\[auth')
        matches_auth = pattern_auth.finditer(resp_seq.text)

        positions = [match.start() for match in matches_auth]
        dic_name = {}
        if positions != []:
            # This means that there are changes of the chain name
            for _ in positions:
                # if test_mode:
                    # print(resp_seq.text[_-2], resp_seq.text[_+6])
                pdb_name = resp_seq.text[_+6]
                cif_name = resp_seq.text[_-2]
                if pdb_name not in dic_name and cif_name >= 'A' and cif_name <= 'Z':
                    print(pdb_name, cif_name)
                    dic_name[pdb_name] = cif_name
        print(dic_name)
        with open(f"/home/t-kaiyuangao/Workspace/DynamicBind/PDBBindData/PDBBind_processed/{pdb_code}/{pdb_code}_protein_processed.pdb", "r") as f:
            # Read the first line which contains chains selected
            lines = f.readlines()
            chains = [line.split()[4][0] for line in lines if line.startswith('ATOM')]
            chains = list(set(chains))
            # chains_tmp = []
            # for idx, chain in enumerate(chains):
            #     if len(chain) > 1: # This means that there are string like ... or

            #         las = chain[-1]
            #         fir = chains[idx - 1][0]
                    
                    
            #         for t in range(ord(fir) + 1, ord(las) + 1):
            #             chains_tmp.append(chr(t))
            #     else:
            #         chains_tmp.append(chain)
            # chains = chains_tmp
            if test_mode:
                print("chains: ", chains)
                print("dic_name: ", dic_name)
            # Now we know the chains contained in the pdb file
            for idx, chain in enumerate(chains):
                if chain in dic_name:
                    chains[idx] = dic_name[chain]
            for idx, chain_list in enumerate(list_assembly_all):
                flag = True
                for chain in chains:
                    if chain not in chain_list:
                        flag = False
                        break
                if flag:
                    # Find the matched chain
                    checked = idx
                    break
    if not flag and len(list_assembly_all) > 1:
        print(f"Warning: {pdb_code} has no matched assembly chains, use the first one")
    return checked

In [21]:
sequence_url = f"https://www.rcsb.org/annotations/1fd7"
resp_seq = requests.get(sequence_url)
with open(f"test.html", "w") as f:
    f.write(resp_seq.text)


In [ ]:
import rcsb
rcsb.find_assembly_index()

In [31]:
import os
with open('PDBBindData/failed_assembly_choose_1.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        for i in range(len(line)):
            with open('PDBBindData/failed_assembly_choose_0.txt', 'a') as f:
                f.write(f"{line[i]}")
            if i % 4 == 3:
                with open('PDBBindData/failed_assembly_choose_0.txt', 'a') as f:
                    f.write('\n')

        print(line)

6p7p4g8o1ow44lkg4q3u1dzp5jzi4p5d1cea3nba3kd73nik1jyc6dyw4zyq3tvw6p7q2r3c1uys1w2x2e9v5cbm4avg5bjt4q3t4uuq4fci6r181szd3w545c4l5vnb4asj2jb64zy15jyp5t2y3lqi4aa71b2m4cku3ubx3zlk2ydw3f811u9l3kr52yem4k7n4ory4nl13b951pau1m214uzd1g6r6ee22ybp6eir1z9h3o5n3h914zy03tv53cii2fw64zx94q3r3kr43twd4a1w1jyi4hww4b4g3h2o3lp73t6r4b3u3wtn4zy23h0s5ctb4b4m6dyu3nim6r1w3sjt2i473lp46r134zx84k3n4hxq4l4v3f805vh03nil4dsy6eee4b5b4u0g3n8k6qau5o876r1c2qtr4r6t3ti84e8y5elw5doh6dyv6o503h0j4q3s1y3a5ctc4n073h0q3t8w6bkx5u493zst6r126c5f1wc16bt06epy6o4w1ezf6mia1kc7


In [ ]:

with open('PDBBindData/failed_assembly_choose_1.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        print(line)



In [ ]:
tmp = []
for _ in lines:
    tmp.append(_[:-1])
    print(_[:-1])

In [49]:
tmp

['6p7p',
 '4g8o',
 '1ow4',
 '4lkg',
 '4q3u',
 '1dzp',
 '5jzi',
 '4p5d',
 '1cea',
 '3nba',
 '3kd7',
 '3nik',
 '1jyc',
 '6dyw',
 '4zyq',
 '3tvw',
 '6p7q',
 '2r3c',
 '1uys',
 '1w2x',
 '2e9v',
 '5cbm',
 '4avg',
 '5bjt',
 '4q3t',
 '4uuq',
 '4fci',
 '6r18',
 '1szd',
 '3w54',
 '5c4l',
 '5vnb',
 '4asj',
 '2jb6',
 '4zy1',
 '5jyp',
 '5t2y',
 '3lqi',
 '4aa7',
 '4cku',
 '3ubx',
 '3zlk',
 '2ydw',
 '3f81',
 '1u9l',
 '3kr5',
 '2yem',
 '4k7n',
 '4ory',
 '4nl1',
 '3b95',
 '1pau',
 '1m21',
 '4uzd',
 '1g6r',
 '6ee2',
 '2ybp',
 '6eir',
 '1z9h',
 '3o5n',
 '3h91',
 '4zy0',
 '3tv5',
 '3cii',
 '2fw6',
 '4zx9',
 '4q3r',
 '3kr4',
 '3twd',
 '4a1w',
 '1jyi',
 '4hww',
 '4b4g',
 '3h2o',
 '3lp7',
 '3t6r',
 '4b3u',
 '3wtn',
 '4zy2',
 '3h0s',
 '5ctb',
 '4b4m',
 '6dyu',
 '3nim',
 '6r1w',
 '3sjt',
 '2i47',
 '3lp4',
 '6r13',
 '4zx8',
 '4k3n',
 '4hxq',
 '4l4v',
 '3f80',
 '5vh0',
 '3nil',
 '4dsy',
 '6eee',
 '4b5b',
 '4u0g',
 '3n8k',
 '6qau',
 '5o87',
 '6r1c',
 '2qtr',
 '4r6t',
 '3ti8',
 '4e8y',
 '5elw',
 '5doh',
 '6dyv',
 

In [ ]:
for _ in tmp:
    

In [90]:
folder_path = "PDBBindData/PDBBind_processed"
# import PDBBindrcsb
pdb_codes = [name for name in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, name))]
oper_more_than_one = []
for pdb_code in pdb_codes:
    try:
        cif_path = f'/home/t-kaiyuangao/Workspace/DynamicBind/PDBBindData/.tmp/{pdb_code}.cif'
        doc = cif.read_file(cif_path)
        block = doc.sole_block()
        # Find assembly Chains
        if len(list(list(block.find_values('_pdbx_struct_assembly_gen.oper_expression'))[0])) > 1:
            oper_more_than_one.append(pdb_code)
            print(pdb_code)
            print(len(oper_more_than_one))
    except:
        continue

2qn2
1
3eqb
2
2fmb
3
5cuu
4
5l4j
5
3ti6
6
6mmc
7
6hkz
8
1joj
9
1mau
10
5kqd
11
4ipj
12
2gz8
13
2cmf
14
4oc5
15
5l17
16
3nm6
17
3fkt
18
2fqo
19
4leq
20
4mwr
21
3t4p
22
2qnb
23
4fck
24
4hiq
25
6g84
26
1grp
27
3il6
28
5fnc
29
3sl1
30
4m3f
31
1a9q
32
1jmg
33
4lkg
34
6dyz
35
4d8a
36
5ai6
37
2jbk
38
5n8w
39
2qrp
40
1l6s
41
1ugx
42
5knr
43
3n1w
44
4lno
45
3o1d
46
4dq2
47
4q3u
48
1vj5
49
2qve
50
1m1b
51
4dwg
52
5opc
53
4ezq
54
4jvj
55
4nan
56
5l2n
57
5alm
58
2cll
59
5g5f
60
1pwy
61
1f8e
62
1g9t
63
4p4j
64
3tz2
65
5e0l
66
1qca
67
3v3m
68
3pck
69
2yc5
70
6d4u
71
3rt8
72
6oag
73
5w2p
74
2azb
75
1lyx
76
6aqs
77
3i51
78
6ppy
79
1str
80
6bo6
81
5c2o
82
1ols
83
3sjf
84
5fnd
85
6g9i
86
5l2m
87
3d7d
88
5w6u
89
4xrq
90
5diq
91
2w8j
92
4dhq
93
6jof
94
3mfv
95
2zx8
96
4ezt
97
2p09
98
5mro
99
3uec
100
5khg
101
1zp8
102
5j4v
103
3amv
104
5am5
105
4p5d
106
5w5u
107
2a8g
108
4bt3
109
2a5c
110
2y76
111
4fs3
112
5zhl
113
1vcj
114
4lrr
115
3c5u
116
5alr
117
4lkk
118
4eky
119
2wa4
120
2xdl
121
6idb
122
6aao
123
4

FileNotFoundError: [Errno 2] No such file or directory: 'Nonredund_annotations.csv'